# Finite State Machines in Digital Design

*From state diagrams to hardware implementation*

---

## Prerequisites

This tutorial assumes you have completed:

- **D Flip-Flops in Digital Design** — understanding edge-triggered storage and clock signals
- **Timing Diagrams in Digital Design** — reading waveforms and understanding signal timing

You should also be familiar with:

- **Binary numbers** and basic Boolean algebra (AND, OR, NOT)
- **Logic gates** and how to combine them into circuits

---

## What is a Finite State Machine?

Finite State Machines (FSMs) are fundamental building blocks in digital design. They model systems that can be in one of a finite number of states, transitioning between states based on inputs.

An FSM consists of:

- **States**: A finite set of conditions the system can be in
- **Inputs**: External signals that influence state transitions
- **Outputs**: Signals produced by the machine
- **Transitions**: Rules for moving between states based on inputs
- **Initial State**: The starting state when the system powers on

Think of a traffic light: it cycles through states (Red → Green → Yellow → Red), with timing inputs controlling transitions and the light color as output.

---

## State Diagrams: Visualizing FSMs

State diagrams use circles for states and arrows for transitions. Let's visualize a simple two-state FSM that toggles between ON and OFF based on a button press.

This is a **Moore machine** — the output (LED on/off) depends only on the current state.

:::{note}
**About the "button" input:** In FSMs, all inputs are **digital signals synchronized to the clock**. The "button" here represents a clean 0/1 signal sampled at each clock edge, not a raw physical button (which would be asynchronous and bouncy). In practice, this signal would come from button debouncing and edge-detection circuitry.
:::

**Color convention:** Green = output HIGH (LED on), Grey = output LOW (LED off)

In [ ]:
from graphviz import Digraph
from IPython.display import display, SVG

dot = Digraph('toggle_fsm')
dot.attr(rankdir='LR')
dot.attr('node', shape='circle', style='filled', fontname='Arial', fontsize='12', width='0.8')
dot.attr('edge', fontname='Arial', fontsize='10')

dot.node('OFF', 'OFF\nLED=0', fillcolor='#6b7280', fontcolor='white')
dot.node('ON', 'ON\nLED=1', fillcolor='#10b981', fontcolor='white')

dot.node('start', '', shape='point', width='0')
dot.edge('start', 'OFF')

dot.edge('OFF', 'ON', label='button=1', color='#3b82f6', fontcolor='#3b82f6', penwidth='2')
dot.edge('ON', 'OFF', label='button=1', color='#3b82f6', fontcolor='#3b82f6', penwidth='2')

dot.edge('OFF', 'OFF', label='button=0', color='#9ca3af', fontcolor='#6b7280')
dot.edge('ON', 'ON', label='button=0', color='#9ca3af', fontcolor='#6b7280')

display(SVG(dot.pipe(format='svg')))

### State Table

| Current State | Input (button) | Output (LED) | Next State |
|--------------|----------------|--------------|------------|
| OFF          | 0              | 0            | OFF        |
| OFF          | 1              | 0            | ON         |
| ON           | 0              | 1            | ON         |
| ON           | 1              | 1            | OFF        |

**Key insight:** The output shown is what the current state produces. After the clock edge, you enter the next state and its output becomes active.

---

## Moore vs Mealy Machines

| Aspect | Moore | Mealy |
|--------|-------|-------|
| **Output depends on** | Current state only | Current state AND inputs |
| **Timing complexity** | Simpler | More complex |
| **Output changes** | Only on clock edges | Can change with inputs |



### Example: Sequence Detector

Detecting when input X has been 1 for two consecutive clock cycles. Output Z goes HIGH when we've seen two 1s in a row.

In [ ]:
from graphviz import Digraph
from IPython.display import display, HTML

moore = Digraph('moore')
moore.attr(rankdir='LR', label='Moore Machine', labelloc='t', fontsize='14')
moore.attr('node', shape='circle', style='filled', fontname='Arial', fontsize='11', width='0.7')
moore.node('S0', 'S0\nZ=0', fillcolor='#3b82f6', fontcolor='white')
moore.node('S1', 'S1\nZ=0', fillcolor='#3b82f6', fontcolor='white')
moore.node('S2', 'S2\nZ=1', fillcolor='#10b981', fontcolor='white')
moore.node('start_m', '', shape='point', width='0')
moore.edge('start_m', 'S0')
moore.edge('S0', 'S1', label='X=1')
moore.edge('S1', 'S2', label='X=1')
moore.edge('S2', 'S2', label='X=1')
moore.edge('S0', 'S0', label='X=0')
moore.edge('S1', 'S0', label='X=0')
moore.edge('S2', 'S0', label='X=0')

moore_svg = moore.pipe(format='svg').decode('utf-8')
display(HTML(f'<div>{moore_svg}</div>'))

---

## State Encoding

To implement an FSM in hardware, we assign binary codes to states. 

### Step 1: Encoded State Table

| Q1 | Q0 | X | D1 (Next Q1) | D0 (Next Q0) | Z |
|----|----|---|--------------|--------------|---|
| 0  | 0  | 0 | 0            | 0            | 0 |
| 0  | 0  | 1 | 0            | 1            | 0 |
| 0  | 1  | 0 | 0            | 0            | 0 |
| 0  | 1  | 1 | 1            | 0            | 0 |
| 1  | 0  | 0 | 0            | 0            | 1 |
| 1  | 0  | 1 | 1            | 0            | 1 |
| 1  | 1  | X | -            | -            | - |

### Step 2: Karnaugh Maps

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np

plt.rcParams.update({
    "text.usetex": False,
    "mathtext.fontset": "cm",
})

fig, axes = plt.subplots(1, 3, figsize=(18, 7))

d1_data = np.array([[0, 0], [0, 1], ['-', '-'], [0, 1]])
d0_data = np.array([[0, 1], [0, 0], ['-', '-'], [0, 0]])
z_data = np.array([[0, 0], [0, 0], ['-', '-'], [1, 1]])

def draw_kmap(ax, data, title, groupings):
    ax.set_xlim(-0.5, 2.5); ax.set_ylim(-1.2, 5)
    ax.set_aspect('equal'); ax.axis('off')
    ax.set_title(title, fontsize=16, fontweight='bold')
    
    for x_idx, y_idx, w, h, color, label in groupings:
        rect = Rectangle((x_idx, y_idx), w, h, color=color, alpha=0.2, linestyle='--')
        ax.add_patch(rect)
        if label: ax.text(x_idx + w/2, y_idx + h/2, label, color=color, ha='center')

    ax.text(-0.3, 4.3, r'$Q_1Q_0$', ha='right')
    row_labels = ['00', '01', '11', '10']
    for i, label in enumerate(row_labels):
        ax.text(-0.3, 3.5 - i, label, ha='right')

    for i in range(4):
        for j in range(2):
            rect = Rectangle((j, 3-i), 1, 1, fill=False)
            ax.add_patch(rect)
            ax.text(j + 0.5, 3.5 - i, str(data[i][j]), ha='center')

draw_kmap(axes[0], d1_data, r'$D_1$ (Next $Q_1$)', [(1, 1, 1, 1, 'blue', r'$Q_0X$'), (1, 0, 1, 1, 'green', r'$Q_1X$')])
draw_kmap(axes[1], d0_data, r'$D_0$ (Next $Q_0$)', [(1, 3, 1, 1, 'orange', r'$\overline{Q_1}\overline{Q_0}X$')])
draw_kmap(axes[2], z_data, r'$Z$ (Output)', [(0, 0, 2, 1, 'red', r'$Q_1$')])

plt.show()

## Key Takeaways

1. **FSMs** are built from Combinational Logic and Sequential Flip-Flops.
2. **Moore Machines** are more stable as outputs only change on clock edges.
3. **K-Maps** help minimize the gates needed for the Next-State logic.